<a href="https://colab.research.google.com/github/GyeeunJeong/Hands-on-ML/blob/main/ch6_decisiontree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 결정 트리 학습과 시각화

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:]
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

In [2]:
# 훈련된 결정트리 시각화
from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file='iris_tree.dot',
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

In [3]:
!dot -Tpng iris_tree.dot -o iris_tree.png

## 예측하기
* 루트 노드: 깊이가 0인 맨 꼭대기의 노드
* 자식 노드
* 리프 노드: 자식 노드를 가지지 않는 노드
* 장점: 데이터 전처리가 거의 필요하지 않다, 화이트박스
* 노드의 gini 속성은 불순도를 측정
  - 지니불순도: $G_i=1-Σ^n_{k=1}p_{i,k}^2$
  - 엔트로피를 사용하기도 하는데 지니불순도 계산이 더 빠름
  - 큰 차이는 없지만 엔트로피가 조금 더 균형 잡힌 트리를 만듬
  - 엔트로피: $H_i=-\displaystyle\sum ^n_{k=1, p_{i,k}\neq0}p_{i,k}log_2(p_{i,k})$

* 훈련 데이터에 대한 제약사항이 거의 없음
  - 비파라미터 모델: 훈련되기 전에 파라미터 수가 결정되지 않음
  - 과대적합될 가능성 높음 (자유도 제한 필요 = 규제 필요)
* 가지치기(pruning): 제한 없이 결정 트리를 훈련하고 불필요한 노드 제거
  - 순도 높이는 것이 통계적으로 효과가 없다면 리프 노드 바로 위의 노드는 불필요할 수 있음 (카이 검정 등을 사용하여 추정)
  - 불필요한 노드가 모두 없어질 때까지 계속됨

* 불안정성
  - 계단 모양의 결정 경계를 만들어 훈련 세트의 회전에 민감함
  - PCA 기법을 사용해 더 좋은 방향으로 회전시킬 수 있음
  

## 클래스 확률 추정
* 한 샘플이 특정 클래스 k에 속할 확률 추정
1. 이 샘플에 대한 리프 노드를 찾기 위해 트리를 탐색
2. 그 노드에 있는 클래스 k의 훈련 샘플의 비율을 반환

In [4]:
tree_clf.predict_proba([[5, 1.5]]) #0/54 49/54 5/54

array([[0.        , 0.90740741, 0.09259259]])

In [5]:
tree_clf.predict([[5, 1.5]])

array([1])

## CART 훈련 알고리즘
* Classification And Regression Tree
1. 훈련 세트를 하나의 특성 k의 임곗값 $t_k$를 사용해 두 개의 서브셋으로 나눔
  - 가장 순수한 서브셋으로 나눌 수 있는 $(k, t_k)$ 짝을 찾음
  - $J(k,t_k)=\frac{m_{left}}{m}G_{left}+\frac{m_{right}}{m}G_{right}$
  - $G_{left/right}$: 왼쪽/오른쪽 서브셋의 불순도
  - $m_{left/right}$: 왼쪽/오른쪽 서브셋의 샘플 수
2. 같은 방식으로 서브셋을 계속 나눔
3. 최대 깊이(max_depth 매개변수로 정의)가 되면 중지하거나 불순도를 줄이는 분할을 찾을 수 없을 때 멈춤
  - 중지 조건에 관여하는 매개변수: min_samples_split, min_samples_leaf, min_weight_fraction_leaf, max_leaf_nodes
* greedy 알고리즘
* 훈련 복잡도: O(n*mlog(m)). m:훈련 데이터 수, n: 특성수


## 회귀
* 각 노드에서 클래스를 예측하는 대신 어떤 값을 예측
* CART는 평균제곱오차(MSE)를 최소화하도록 분할
  - $J(k, t_k)=\frac{m_{left}}{m}MSE_{left}+\frac{m_{right}}{m}MSE_{right}$


In [6]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

DecisionTreeRegressor(max_depth=2)